# 如何处理未生成查询的情况

:::info 前提条件

本指南假定您熟悉以下内容：

- [查询分析](/docs/tutorials/rag#query-analysis)

:::

有时，一种查询分析技术可能会生成任意数量的查询——甚至不生成任何查询！在这种情况下，我们的整体链需要先检查查询分析的结果，再决定是否调用检索器。

本例中我们将使用模拟数据。

## 安裝

### 安裝依賴

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/openai @langchain/core zod chromadb
</Npm2Yarn>
```

### 設定環境變數

```
OPENAI_API_KEY=your-api-key

# 可選，使用 LangSmith 以獲得最佳的可觀測性
LANGSMITH_API_KEY=your-api-key
LANGSMITH_TRACING=true

# 如果你不在無伺服器環境中，可減少追蹤延遲
# LANGCHAIN_CALLBACKS_BACKGROUND=true
```

### 创建索引

我们将在虚假信息上创建一个向量存储。

In [ ]:
import { Chroma } from "@langchain/community/vectorstores/chroma"
import { OpenAIEmbeddings } from "@langchain/openai"
import "chromadb";

const texts = ["Harrison worked at Kensho"]
const embeddings = new OpenAIEmbeddings({ model: "text-embedding-3-small" })
const vectorstore = await Chroma.fromTexts(texts, {}, embeddings, {
  collectionName: "harrison"
})
const retriever = vectorstore.asRetriever(1);

## 查询分析

我们将使用函数调用来构建输出结构。但是，我们会配置大语言模型（LLM），使其不需要调用代表搜索查询的函数（如果它决定不调用的话）。此外，我们还会使用一个提示词来进行查询分析，明确说明何时应该以及不应该执行搜索。

In [24]:
import { z } from "zod";

const searchSchema = z.object({
    query: z.string().describe("Similarity search query applied to job record."),
});

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [ ]:
// @lc-docs-hide-cell
import { ChatOpenAI } from '@langchain/openai';

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

In [25]:
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableSequence, RunnablePassthrough } from "@langchain/core/runnables";

const system = `You have the ability to issue search queries to get information to help answer user information.

You do not NEED to look things up. If you don't need to, then just respond normally.`;
const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", system],
    ["human", "{question}"],
  ]
)
const llmWithTools = llm.bind({
  tools: [{
    type: "function" as const,
    function: {
      name: "search",
      description: "Search over a database of job records.",
      parameters: zodToJsonSchema(searchSchema),
    }
  }]
})
const queryAnalyzer = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
  },
  prompt,
  llmWithTools
])

我们可以看到，通过调用此方法，我们得到一条消息，该消息有时（但并非总是）会返回一个工具调用。

In [26]:
await queryAnalyzer.invoke("where did Harrison work")

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_uqHm5OMbXBkmqDr7Xzj8EMmd",
          type: "function",
          function: [Object]
        }
      ]
    }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_uqHm5OMbXBkmqDr7Xzj8EMmd",
        type: "function",
        function: { name: "search", arguments: '{"query":"Harrison"}' }
      }
    ]
  }
}

In [27]:
await queryAnalyzer.invoke("hi!")

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

## 使用查询分析进行检索

那么我们如何在链中包含这一点呢？让我们看一下下面的示例。

In [33]:
import { JsonOutputKeyToolsParser } from "@langchain/core/output_parsers/openai_tools";

const outputParser = new JsonOutputKeyToolsParser({
  keyName: "search",
})

In [46]:
import { RunnableConfig, RunnableLambda } from "@langchain/core/runnables";

const chain = async (question: string, config?: RunnableConfig) => {
  const response = await queryAnalyzer.invoke(question, config);
  if ("tool_calls" in response.additional_kwargs && response.additional_kwargs.tool_calls !== undefined) {
    const query = await outputParser.invoke(response, config);
    return retriever.invoke(query[0].query, config);
  } else {
    return response;
  }
}

const customChain = new RunnableLambda({ func: chain });

In [47]:
await customChain.invoke("where did Harrison Work")

[ Document { pageContent: "Harrison worked at Kensho", metadata: {} } ]

In [48]:
await customChain.invoke("hi!")

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

## 下一步
您现在已经了解了一些在查询分析系统中处理无关问题的技术。

接下来，可以查看本节中其他一些查询分析指南，例如[如何使用少样本示例](/docs/how_to/query_few_shot)。